###  MicroGrad demo

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pprint
from mycrograd_debug.engine_debug import Value
from mycrograd_debug.nn_debug import Neuron, Layer, MLP
from mycrograd_debug.drawviz_debug import draw_dot,draw_nn
np.random.seed(1337)
random.seed(1337)
pp = pprint.PrettyPrinter(indent=4)

In [2]:
# initialize a model 
model = MLP(3, [4, 4, 1]) # 2-layer neural network
# print(model)
# print("number of parameters", len(model.parameters()))
pp.pprint(model.parameters())

Module nn MLP: structure [3, 4, 4, 1]
[   Value(name=v001,layernumber=L1,neuronnumber=N1,weightnumber=,type=w1,data=0.23550571390294128, grad=0),
    Value(name=v002,layernumber=L1,neuronnumber=N1,weightnumber=,type=w2,data=0.06653114721000164, grad=0),
    Value(name=v003,layernumber=L1,neuronnumber=N1,weightnumber=,type=w3,data=-0.26830328150124894, grad=0),
    Value(name=v004,layernumber=L1,neuronnumber=N1,weightnumber=,type=b,data=0, grad=0),
    Value(name=v005,layernumber=L1,neuronnumber=N2,weightnumber=,type=w1,data=0.1715747078045431, grad=0),
    Value(name=v006,layernumber=L1,neuronnumber=N2,weightnumber=,type=w2,data=-0.6686254326224383, grad=0),
    Value(name=v007,layernumber=L1,neuronnumber=N2,weightnumber=,type=w3,data=0.6487474938152629, grad=0),
    Value(name=v008,layernumber=L1,neuronnumber=N2,weightnumber=,type=b,data=0, grad=0),
    Value(name=v009,layernumber=L1,neuronnumber=N3,weightnumber=,type=w1,data=-0.23259038277158273, grad=0),
    Value(name=v010,layernum

# Half Moon Example

In [3]:
# make up a dataset

from sklearn.datasets import make_moons, make_blobs

number_of_samples=10
number_of_iterations=10


X, y = make_moons(n_samples=number_of_samples, noise=0.1)
print(X)
print(y)

y = y*2 - 1 # make y be -1 or 1

# initialize a model 
model = MLP(2, [16, 16, 1]) # 2-layer neural network
print(model)
print("number of parameters", len(model.parameters()))

# loss function
def loss(batch_size=None):
    
    # inline DataLoader :)
    if batch_size is None:
        Xb, yb = X, y
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X[ri], y[ri]
    inputs = [list(map(Value, xrow)) for xrow in Xb]
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss
    
    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

total_loss, acc = loss()
print(total_loss, acc)

[[ 0.23716815 -0.17338508]
 [-0.55222318  0.57003313]
 [ 1.14252914 -0.02794639]
 [-1.05596279  0.11863834]
 [ 0.16985189  0.33087798]
 [-0.06995228  1.05829628]
 [ 0.80492904  0.58536957]
 [ 1.86706046  0.49985453]
 [ 1.57564151 -0.24506796]
 [ 1.12652106 -0.48793323]]
[1 0 0 0 1 0 0 1 1 1]
Module nn MLP: structure [2, 16, 16, 1]
MLP of [Layer of [ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2)], Layer of [ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16)], Layer of [LinearNeuron(16)]]
number of parameters 337
Value(name=v7539,layernumber=,neuronnumber=,weightnumber=,type=,data=1.8456615030268986

In [4]:
# optimization
for k in range(number_of_iterations):
    
    # forward
    total_loss, acc = loss()
    
    # backward
    model.zero_grad()
    total_loss.backward()
    
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/number_of_iterations
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    
    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")


step 0 loss 1.8456615030268986, accuracy 30.0%
step 1 loss 0.9025888803542371, accuracy 50.0%
step 2 loss 0.9527683465162705, accuracy 50.0%
step 3 loss 0.7041659714612418, accuracy 70.0%
step 4 loss 0.7379767456259533, accuracy 60.0%
step 5 loss 0.581212108075825, accuracy 70.0%
step 6 loss 0.47789494591370996, accuracy 80.0%
step 7 loss 0.45471480401530767, accuracy 80.0%
step 8 loss 0.5256309335197832, accuracy 70.0%
step 9 loss 0.49265313455711335, accuracy 80.0%
